In [1]:
import numpy as np
import os
import xml.dom.minidom
import xml.etree.cElementTree as ET
import copy
from PIL import Image
from datetime import datetime
import time

In [47]:
class Place():
    
    """
    Eingangsdaten:
    Klassenpfad, Ausgangsbildgrösse in px, Anzahl Bilder Channel
    
    Konstruktor genieriert ein dictonary, welches wie folgt aussieht:
    
    {Pfad_Klassen:{Klassenname_1:{0:Bildname_1, 1:Bildname1}}
                 :{Klassenname_2:{0:Bildname_1, 1:Bildname2}}
    }
    
    2 Varianten wie die einzelen Zeichen platziert werden können:
    
    1. rdm Verteilung
    2. ....
    
    """
    
    def __init__(self, path, xml_path, segemented_image_path, size, depth, iou_threshold):
        self._path = path
        self._xml_path = xml_path
        self._segemented_image_path = segemented_image_path
        self._size = size
        self._depth= depth
        self._iou_threshold = iou_threshold
        self._dictonary = {'path':{'folder_name':{0:'image_name'}}}
        self._segmented_image = np.zeros((self._size, self._size, self._depth))
        self._bounding_boxes = np.array([0,0,0,0])
        self._body_name = ''
        self._date_time = ''
        self._placed_image_path = ''
        self._x_rdm_min = 0
        self._y_rdm_min = 0
        self._x_rdm_max = 0
        self._y_rdm_max = 0
        self._dict_folders = ''
        self._number_dict_folders = 0
        self._number_dict_images = 0
        self._rdm_image = 0
        self._filename = ''
        self._place_trials = 0
        self._result = False
        self._finish = False
        self._placed = False
        
        path_1, dirs_1, files_1 = next(os.walk(self._path))
        self._dictonary = {path_1:{}}
        for folder in dirs_1:
            self._dictonary[path_1][folder] = {}
#             print(folder)
            path_2, dirs_2, files_2 = next(os.walk(path_1 + folder))
            for i, files in zip(range(len(files_2)), files_2):
                self._dictonary[path_1][folder][i] = {files}
#                 print(files) 


    def get_head(self):
#         print('get_head')
        now = datetime.now()
        self._date_time = now.strftime("%Y%m%d_%H%M%S")
        slice_position_r = self._placed_image_path.rfind('/')
        slice_position_l = self._placed_image_path[:slice_position_r].rfind('/')
        self._filename = self._date_time + self._filename
        folder = self._placed_image_path[slice_position_l+1:slice_position_r]
        
        string_head =   '<annotation>'\
                        '<folder>'+str(folder)+'</folder>'\
                        '<filename>'+str(self._filename)+'</filename>'\
                        '<path>'+str(self._placed_image_path)+'</path>'\
                        '<source>'\
                        '<database>Unknown</database>'\
                        '</source>'\
                        '<size>'\
                            '<width>'+str(self._size)+'</width>'\
                            '<height>'+str(self._size)+'</height>'\
                            '<depth>'+str(self._depth)+'</depth>'\
                        '</size>'\
                        '<segmented>0</segmented>'\
                        '</annotation>'
        return string_head

    
    def get_body(self,image_name,xmin,ymin,xmax,ymax):
#         print('get_body')
        tmp_slice_r = self._placed_image_path.rfind('_tailored')
        tmp_slice_l = self._placed_image_path[:tmp_slice_r].rfind('_')
        self._filename = self._filename + '_' + self._placed_image_path[tmp_slice_l+1:tmp_slice_r]
        slice_position = image_name.find('_')
        name = image_name[slice_position-1:slice_position]
        string_object = '<object>'\
                        '<name>'+str(name)+'</name>'\
                        '<pose>Unspecified</pose>'\
                        '<truncated>0</truncated>'\
                        '<difficult>0</difficult>'\
                        '<bndbox>'\
                            '<xmin>'+str(xmin)+'</xmin>'\
                            '<ymin>'+str(ymin)+'</ymin>'\
                            '<xmax>'+str(xmax)+'</xmax>'\
                            '<ymax>'+str(ymax)+'</ymax>'\
                            '</bndbox>'\
                        '</object>'
        return string_object

    
    def iou_check(self):
#         print('iou_check')
        if np.count_nonzero(self._segmented_image) == 0:
            self._bounding_boxes  = np.array([0,0,0,0])
            self._bounding_boxes[0] = self._x_rdm_min
            self._bounding_boxes[1] = self._y_rdm_min
            self._bounding_boxes[2] = self._x_rdm_max
            self._bounding_boxes[3] = self._y_rdm_max
#             print('no iou check')
            self._result = True
        else:
            bounding_box = np.array([0,0,0,0])
            bounding_box[0] = self._x_rdm_min
            bounding_box[1] = self._y_rdm_min
            bounding_box[2] = self._x_rdm_max
            bounding_box[3] = self._y_rdm_max
            length = int(len(self._bounding_boxes)/4)
#             print('place_trials: ', self._place_trials)
            tmp_iou = 0
            tmp_max_iou = 0
            
            for i in range(length):
                tmp_start = 0+(4*i)
                tmp_end = 3+(4*i)
                
#                 print(tmp_start,tmp_end)
                tmp_bounding_boxes = self._bounding_boxes[tmp_start:tmp_end+1]
                tmp_iou = self.bb_intersection_over_union(bounding_box, tmp_bounding_boxes)

                if tmp_iou >= tmp_max_iou:
                    tmp_max_iou = tmp_iou
                    
            
            if tmp_max_iou <= self._iou_threshold: 
                self._bounding_boxes = np.append(self._bounding_boxes,bounding_box)
#                 print('bounding: ',self._bounding_boxes)
                self._place_trials = 0
                self._result = True
            else:
                self._result = False
                if self._place_trials == 3:
                    self.create_xml()
                    self.rdm()
                    
                else:
                    self._place_trials += 1
                    self.place()         
    
    
    def place(self):
#         print('place')
        with Image.open(self._placed_image_path) as image:
            self._img = np.array(image)
            y,x,z = self._img.shape
            x_max = self._size - x - 5
            y_max = self._size - y - 5
            self._x_rdm_min = np.random.randint(5,x_max)
            self._y_rdm_min = np.random.randint(5,y_max)
            self._x_rdm_max = self._x_rdm_min + x
            self._y_rdm_max = self._y_rdm_min + y
            self.iou_check()
            
            y,x,z = (self._segmented_image[self._y_rdm_min:self._y_rdm_max,self._x_rdm_min:self._x_rdm_max,:]).shape
            
            if  self._result == True:
                self._segmented_image[self._y_rdm_min:self._y_rdm_max,self._x_rdm_min:self._x_rdm_max,:] = self._segmented_image[self._y_rdm_min:self._y_rdm_max,self._x_rdm_min:self._x_rdm_max,:] + self._img
                self._placed = True   

        
        
    def create_xml_segmented_image(self, string):
#         print('create_xml_segmented_image')
        dom = xml.dom.minidom.parseString(string)
        pretty_xml_as_string_body = dom.toprettyxml()
        pretty_xml_as_string_body_without_version = pretty_xml_as_string_body[23:]
        filepath = self._xml_path + '/' + self._filename + '.xml'
        with open(filepath, 'w+') as file:
            file.write(pretty_xml_as_string_body_without_version)
            
        
        img_uint8 = Image.fromarray(np.uint8(self._segmented_image))
        img_uint8.save(self._segemented_image_path +'/'+self._filename+'.bmp')
        self._segmented_image = np.zeros((self._size, self._size, self._depth))
        self._filename = ''
            
        
    def create_xml_body(self,):
#         print('create_xml_body')
        slice_position = self._placed_image_path.find('_')
        object_name = self._placed_image_path[:slice_position]
        xml_name_tmp = self.get_body(self._placed_image_path,self._x_rdm_min-1,self._y_rdm_min-1,self._x_rdm_max+1,self._y_rdm_max+1)
        self._body_name = self._body_name + xml_name_tmp
#         print(self._body_name)
              
    def create_xml(self):
#         name = str(Place.placeIt_counter) + 
#         print(xml_head)
        if  self._place_trials == 3 or len(list(self._dictonary[self._path])) == 0:
            xml_head = self.get_head()
            slice_position = xml_head.find('</annotation>')
            complete_xml = xml_head[:slice_position] + self._body_name + xml_head[slice_position:]
    #         print(complete_xml)
            self.create_xml_segmented_image(complete_xml)
            self._body_name = ''  
            self._place_trials = 0
        else:
            self.rdm()
    
    def delete_dict(self): 
#         print('delete')
        del self._dictonary[self._path][self._dict_folders][self._rdm_image]
        if len(list(self._dictonary[self._path][self._dict_folders]))== 0:
            del self._dictonary[self._path][self._dict_folders]
        if len(list(self._dictonary[self._path])) == 0:
            self._finish =  True
        else:
            self._finish =  False
    
    
    def rdm(self):
#         print('rdm')
        ### Zufälliger Ordner von Zeichen z.B. A oder B ###
        self._dict_folders = list(self._dictonary[self._path].keys())
        self._number_dict_folders = len(self._dict_folders)
        
#         print(self._dictonary)
        
        if self._number_dict_folders != 0:
            rdm_folder = np.random.randint(self._number_dict_folders)
            self._dict_folders = self._dict_folders[rdm_folder]
            
            ### Zufälliges Bild von oben bestimmten Zeichen ###
            dict_images = list(self._dictonary[self._path][self._dict_folders].keys())
            
            self._number_dict_images = len(dict_images)
            rdm_element = np.random.randint(self._number_dict_images)
            self._rdm_image = dict_images[rdm_element]
            dict_image_name = str(self._dictonary[self._path][self._dict_folders][self._rdm_image])

            r_tmp = dict_image_name.find('}')
            dict_image_name = dict_image_name[2:r_tmp-1]


            ### Bild Pfad des Zeichens ###
            self._placed_image_path = self._path + self._dict_folders + '/' +  dict_image_name
            self.place()

        if self._placed == True:
            self.create_xml_body() 
            if len(list(self._dictonary[self._path])) != 0:
                self.delete_dict()
                self.create_xml()
                
    
    
    def bb_intersection_over_union(self, boxA, boxB):
#         print('bb_intersection_over_union')
        
        # determine the (x, y)-coordinates of the intersection rectangle
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        # compute the area of intersection rectangle
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

        # compute the area of both the prediction and ground-truth
        # rectangles
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)

#         print('iou: ', iou)
        return iou    


In [48]:
# obj33 = Place('C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/','C:/Users/User/Desktop/Test/GeneratedData/XML_Files/','C:/Users/User/Desktop/Test/GeneratedData/SegmentedImages',512,3,0)

In [49]:
# obj33.rdm()